# Hashtag

script below is used to split hashtag into seperate words
; basic strucutre is to look at all lower case words vs words with at least one upper case
* lower: use a function found from outside source (http://stackoverflow.com/questions/8870261/how-to-split-text-without-spaces-into-list-of-words)
* upper: use custom regex which spits on capital letters

In [2]:
## import all necessary packages
import json
import re

import nltk, string
from nltk.collocations import *

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from pandas import *

from collections import defaultdict

from happyfuntokenizing import TweetTokenizer
# from nltk.tokenize import word_tokenize

import data_cleaning_hr as tc

from math import log

** Load clean json file from notebook **

In [3]:
clean_tweets_df = tc.loader('./data/tweets_training.json')

In [4]:
tweet_df = pd.DataFrame(clean_tweets_df)

In [5]:
tweet_df.head()

,id,lat,lng,text,timeStamp,user_id
0,383560784879833088,37.369106,-121.998494,"I'm at SEPTEMBER♥ (Sunnyvale, CA) http://t.co/ICHmlfq3FR",Fri Sep 27 11:56:31 +0000 2013,47880944
1,385148713460453376,37.002203,-122.058052,oops. fell asleep in the library. class started at 2.,Tue Oct 01 21:06:23 +0000 2013,259008472
2,380062755639676928,37.438669,-121.879072,@nteimoorian you told me your birthday yourself 2chainz lol,Tue Sep 17 20:16:36 +0000 2013,630468267
3,384797380198928384,37.299621,-121.982024,She throw it back like Thursday,Mon Sep 30 21:50:19 +0000 2013,272108001
4,383691693767143425,39.137889,-121.658646,Not ready for this exam,Fri Sep 27 20:36:43 +0000 2013,790458278


** addHashtag function create new column with only hashtags **

In [9]:
def addHashTag(df):
    df['only_HashTag'] = [re.findall(r"(#\w+)", word) for word in df.text]

In [10]:
addHashTag(tweet_df)

In [11]:
tweet_df[10:50]

,id,lat,lng,text,timeStamp,user_id,only_HashTag
10,380511446926635008,37.376171,-122.030318,Time for a splurge with the co-workers. (@ DishDash Restaurant w/ 2 others) http://t.co/uLekECjqwA,Thu Sep 19 01:59:33 +0000 2013,777228427,[]
11,380852618685210625,38.542626,-121.759044,"Ah, the ARC during the summer. Where one knows not if the groans are from strenuous exercise or the pain of hangovers past.",Fri Sep 20 00:35:14 +0000 2013,834886554,[]
12,381957258436816896,33.741023,-117.984205,I'm really obsessed with my toes bc I'm blessed with pretty feet. \n\nIs that conceited ? It's just my feet idk.,Mon Sep 23 01:44:41 +0000 2013,926232458,[]
13,383360559250296832,37.724169,-122.484250,Fucking another fire drill. I'm doneeee,Thu Sep 26 22:40:54 +0000 2013,50575737,[]
14,384443607379243008,37.617409,-122.380052,"Some @ericries reading for the flight. Oh, and thanks @VirginAmerica for the in-flight music! http://t.co/28P7gVuFm3",Sun Sep 29 22:24:33 +0000 2013,18095878,[]
15,381880837911748608,37.698506,-122.144923,Yay for Fall!,Sun Sep 22 20:41:01 +0000 2013,15538455,[]
16,382659372649676800,36.929559,-121.769604,@AlondraDiamond yeahhh ROAD TRIP,Wed Sep 25 00:14:38 +0000 2013,77063592,[]
17,385855209102012416,37.578380,-122.347479,Crepevine with @Kkeellyycc_ 😍 http://t.co/8B0gqgmQWZ,Thu Oct 03 19:53:45 +0000 2013,548146276,[]
18,384476775369039872,37.794656,-122.276863,"Well, that was a unfortunate time not to have a cigarette lighter. Maybe I should just have one around even though I gave up smoking.",Mon Sep 30 00:36:21 +0000 2013,14622512,[]
19,380408506068660225,38.337607,-122.677698,Didn't wanna leave my room for 56 hours anyways ?? No prob! http://t.co/cT5cCnNuat,Wed Sep 18 19:10:30 +0000 2013,622968626,[]


** for purpose of notebook, create list of words with only hashtag, then take out # **

In [59]:
complete_hashtags = [word.split('#')[1] for item in tweet_df.only_HashTag for word in item if len(item) > 0 ]

In [66]:
# print(complete_hashtags[100:200])

** in order to sepearate long string of words into seperate words, must create bag of words and their respective frequencies **

In [18]:
def tokenize_total_text():
    tokenizer = TweetTokenizer()
    total_text = tweet_df.text.apply(lambda x: tokenizer.tokenize(x))
    
    stop_words = nltk.corpus.stopwords.words('english') + ["http", 'hdl', 'url'] 
    punctuation_words = list(set(string.punctuation)) + [":", ":/"]

    total_text = list(total_text)
    total_text = [word for _list in total_text for word in _list if word not in stop_words and word not in punctuation_words]
    
    return total_text

In [19]:
total_text = tokenize_total_text()

In [20]:
total_text[:10]

["i'm",
 'september',
 '♥',
 'sunnyvale',
 'ca',
 'http://t.co/ICHmlfq3FR',
 'oops',
 'fell',
 'asleep',
 'library']

** use frequency of words from above to create infer_spaces function for 'lowercase' words **

In [21]:
# words = open("words-by-frequency.txt").read().split()
wordcost = dict((k, log((i+1)*log(len(total_text)))) for i,k in enumerate(total_text))
maxword = max(len(x) for x in total_text)

In [482]:
def infer_spaces(s):
    """Uses dynamic programming to infer the location of spaces in a string
    without spaces."""
    
    # Build a cost dictionary, assuming Zipf's law and cost = -math.log(probability).

    # Find the best match for the i first characters, assuming cost has
    # been built for the i-1 first characters.
    # Returns a pair (match_cost, match_length).
    def best_match(i):
        candidates = enumerate(reversed(cost[max(0, i-maxword):i]))
        return min((c + wordcost.get(s[i-k-1:i], 9e999), k+1) for k,c in candidates)

    # Build the cost array.
    cost = [0]
    for i in range(1,len(s)+1):
        c,k = best_match(i)
        cost.append(c)

    # Backtrack to recover the minimal-cost string.
    out = []
    i = len(s)
    while i>0:
        c,k = best_match(i)
        assert c == cost[i]
        out.append(s[i-k:i])
        i -= k

    return list(reversed(out))

In [483]:
s = 'onlyone'
print(infer_spaces(s))

['onlyone']


** for words with at least one uppercase, use custom regex **

In [484]:
def convert_hashtag(_hashtag):    
#     pattern = r'([A-Z]+[a-z]*)|([A-Z]{2,})'
#     pattern = r"""([A-Z]+[a-z]*)"""
#     pattern = r"""([A-Z]{2,})"""
    pattern = r'([A-Z]{2,})|([A-Z]{1}[a-z]*)'

    for word in _hashtag:
        if len(word) == 0:
            print("no hashtag")
        elif word[0].isupper() and word[1:].islower():
            print("lower_forced: " + word + " : " + infer_spaces(word.lower()))
        elif word.islower():
            print("lower       : " + word + " : " + infer_spaces(word))
        else:
            print("upper       : " + word + " : ", end="")
            print(list(filter(None, re.split(pattern, word))))

        
# example_hashtags

In [485]:
# convert_hashtag(complete_hashtags[400:500])

In [486]:
# example_hashtags = ["SFgiants","10HelloWorld","Thereispeaceinthenextlife","beGLOBAL","TBT","HELLOoo","GREATmusic","Top10"]

In [487]:
# convert_hashtag(example_hashtags)

In [536]:
def apply_hashtag_split(_hashtag):
    # for each hashtag in list of hashtags, split on # and take second item
    item = [item.split('#')[1] for item in _hashtag if len(item) > 0]
    
    # regex pattern
    pattern = r'''([A-Z]{2,})      | (1) split on two caps or more, and only keep caps
                  ([A-Z]{1}[a-z]*)   (2) split on exactly one cap or more, and keep 
                                         trailing letters '''
    
    # loop through each item in list of hashtags
    final_hashtags = []
    for word in item:
        
        # if len of word is 0, then there is no hashtag
        if len(word) == 0:
            print("empty_hashtag")
            final_hashtags.append("empty_hashtag")
        
        # use (1)) regex: funciton lowercase, as "Treatlowercase" can be treated as lowercase
        elif word[0].isupper() and word[1:].islower():
            print("lower_forced: " + word + " : ", end="")
            print(infer_spaces(word.lower()))
            final_hashtags.append(infer_spaces(word.lower()))
        
        # use (1) regex: funciton lowercase
        elif word.islower():
            print("lower       :: " + word + " : ", end="")
            print(infer_spaces(word))
            final_hashtags.append(infer_spaces(word))
        
        # use (2) regex: customized uppercase
        else:
            print("upper       : " + word + " : ", end="")
            print(list(filter(None, re.split(pattern, word))))
            final_hashtags.append(list(filter(None, re.split(pattern, word))))
    return final_hashtags

In [540]:
apply_hashtag_split(["#",
                     "",
                     "#33DildO89",
                     "#shitnuts",
                     "#Dhello"])

empty_hashtag
upper       : 33DildO89 : ['33DildO89']
lower       :: shitnuts : ['shit', 'nuts']
lower_forced: Dhello : ['dh', 'ello']


['empty_hashtag', ['33DildO89'], ['shit', 'nuts'], ['dh', 'ello']]